# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.236156080297                   -0.50    7.0
  2   -7.250474058433       -1.84       -1.40    1.0
  3   -7.251315384763       -3.08       -2.18    3.0
  4   -7.251263006411   +   -4.28       -2.22    3.0
  5   -7.251333537066       -4.15       -2.79    2.0
  6   -7.251338517409       -5.30       -3.35    2.0
  7   -7.251338784484       -6.57       -4.04    2.0
  8   -7.251338795568       -7.96       -4.29    4.0
  9   -7.251338798414       -8.55       -4.81    2.0
 10   -7.251338798672       -9.59       -5.29    3.0
 11   -7.251338798701      -10.53       -5.87    2.0
 12   -7.251338798704      -11.47       -6.58    3.0
 13   -7.251338798705      -13.33       -7.17    3.0
 14   -7.251338798705      -14.75       -7.41    3.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.08745178558185854
[ Info: Arnoldi iteration step 2: normres = 0.5295427121581607
[ Info: Arnoldi iteration step 3: normres = 0.5691846709165788
[ Info: Arnoldi iteration step 4: normres = 0.34553517912553433
[ Info: Arnoldi iteration step 5: normres = 0.31375925593450976
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (7.70e-03, 7.80e-02, 2.07e-01, 2.21e-01, 2.91e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4729186323709716
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.02e-03, 1.57e-01, 4.22e-01, 9.61e-02, 7.05e-02)
[ Info: Arnoldi iteration step 7: normres = 0.080555133010573
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.29e-04, 2.14e-02, 2.28e-02, 2.69e-02, 6.32e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09910528650003968
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.54e-0